# Exercício 1 - 8-Puzzle

In [1]:
from collections import deque
import time

In [4]:
class Node:
    def __init__(self, state, cost, depth, move = "", parent= None) :
        self.parent = parent
        self.state = state
        self.move = move
        self.cost = cost
        self.depth = depth

In [3]:
class Problem:
    def __init__(self, size, initial_state, goal):
        self.initial_state = initial_state
        self.goal = goal
        self.size = size
    
    
    # Diz se o estado atual chegou ao objetivo.
    def objective_test(self, state):
        return state == self.goal

    # Pega a coordenada x a partir de um index do vetor
    def get_x(self, index):
        return index - self.get_y(index) * self.size

    # Pega a coordenada y a partir de um index do vetor
    def get_y(self, index):
        return index // self.size

    
    # Pega aa coordenadas x e y a partir de um index do vetor
    def get_coords(self, index):
        return self.get_x(index), self.get_y(index)

    
    # Pega o index do vetor a partir das coordenadas x e y
    def get_index(self, x, y):
        return y * self.size + x

    
    # Pega as possíveis ações que o problema pode ter
    def actions(self, state):
        moves = ["Up", "Right", "Left", "Down"]
        index = state.index(0)
        x, y = self.get_coords(index)

        if x == 0:
            moves.remove("Left")
        elif x == self.size - 1:
            moves.remove("Right")

        if y == 0:
            moves.remove("Up")
        elif y == self.size - 1:
            moves.remove("Down")

        return moves

    
    # Faz o movimento 'action' no estado 'estate' e retorna o novo estado
    def move(self, state, action):
        result = state.copy()
        index = state.index(0)
        x, y = self.get_coords(index)

        if action == "Up":
            y -= 1
        elif action == "Down":
            y += 1
        elif action == "Left":
            x -= 1
        elif action == "Right":
            x += 1

        new_index = self.get_index(x, y)

        result[index], result[new_index] = result[new_index], result[index]

        return result

    
    # Pega o path que fez chegar no 'node'
    def get_path(self, node):
        moves = []

        while node.parent:
            moves.append(f"'{node.move}'")
            node = node.parent

        moves.reverse()
        return ", ".join(moves)

In [ ]:
def manhattan_distance(problem:Problem, state):
    result = 0
    x,y,x_s,y_s = 0
    for index in range(problem.size * problem.size):
        x, y = problem.get_coords(index)
        for index_s, item in enumerate(state):
            if item == index:
                x_s, y_s = problem.get_coords(index_s)
                break
        result += abs(x - x_s) + abs(y - y_s)
        
    return result

In [ ]:
def greedy(problem:Problem, heuristic):
    maxdepth = 0
    maxfrontier = 0
    # Criando o primeiro nó
    node = Node(problem.initial_state, 0)
    
    if problem.objective_test(node.state):
        return {"node": node, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": 0, "scanned": 0}
    
    # Criando as listas que vão ser usadas
    edge = deque()
    edge.append(node)
    explored = set()
    
    # Enquanto edge não for vazia
    while edge:
        # Pega o primeiro valor que entrou na fila e adiciona nos explorados
        node = edge.popleft()
        explored.add(str(node.state))
        
        
        actions = sorted(problem.actions(), key = lambda action: heuristic(problem, problem.move(node.state, action)))

        # Pega as possíveis ações que podemos fazer a partir do estado atual
        for action in actions:
            
            # Realiza a ação e pega o novo estado
            newstate = problem.move(node.state, action)
            
            # Cria o filho do nó atual com a ação selecionada
            son = Node(newstate, node.cost + 1, action, node)
            
            # Checa se o filho já foi explorado
            if str(son.state) not in explored:
                # Gravando a variável de maior profundidade
                if son.cost > maxdepth:
                    maxdepth = son.cost

                # Se o filho for a resposta, retorna a solução
                if problem.objective_test(son.state):
                    return {"node": son, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": len(edge), "scanned": len(explored)}
                
                # Adiciona o filho a fila e na lista de explorados
                edge.append(son)
                explored.add(str(son.state))

        # Gravando a variável de maior fronteira
        if len(edge) > maxfrontier: 
            maxfrontier = len(edge)
        
        
    return {"node": None, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": len(edge), "scanned": len(explored)}
